In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
jump_index = np.where((train[:-1,1] - train[1:,1])<0)[0]
jump_index = np.insert(jump_index,0,0)
jump_index = np.append(jump_index,train.shape[0])

In [3]:
## change ##
batch_size = 16
clip = .5

Prepare data

In [71]:
train_gen = SequenceGenNojump(train,jump_index)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenNojump(train,jump_index,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [52]:
convNets = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      DenseBlock(32),
                      DenseBlock(64),
                      Dropout(0.5),
                      DenseBlock(128),
                      Dropout(0.5),
                      DenseBlock(256),
                      Dropout(0.5),
                      GRU_NCL(512,512,batch_first=True,bidirectional=True,returnH=True))

In [53]:
linear = Sequential(Dropout(0.5),Linear(512*2,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [54]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [55]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [56]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [57]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.675713084855126, val_loss:2.778198719024658
epoch:1, train_loss:2.3299542888275626, val_loss:2.5323379039764404
epoch:2, train_loss:2.295732397477604, val_loss:2.736665964126587
epoch:3, train_loss:2.2430119913758584, val_loss:2.3283913135528564
epoch:4, train_loss:2.2078342614243334, val_loss:2.9633941650390625
Training completed in 75.48741006851196s


Submission

In [75]:
submission = pd.read_csv('../Data/sample_submission.csv',)

In [76]:
test_data = []
for a in submission.seg_id.tolist():
    b = pd.read_csv('../Data/test/'+a+'.csv',dtype={'acoustic_data': np.float32})
    b = normalize(b.values.flatten())
    test_data.append(b)
test_data = np.stack(test_data,0)[:,np.newaxis]

In [77]:
test_gen = numpyArray(test_data)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [78]:
submission.iloc[:,1] = predict(model,test_gen)

In [79]:
submission.to_csv('../Submission/sub_noJump.csv',index=False)